In [1]:
import pandas as pd
import numpy as np
import glob

In [9]:
# 1달치 데이터 가져옴
item_list = glob.glob('../rawdata/2014-01*')
for idx, p in enumerate(item_list):
    df = pd.read_csv(p, encoding = 'cp949')
    if idx == 0:
        all_df = df
    else:
        all_df = pd.concat([all_df, df])

In [10]:
all_df.shape

(2800079, 8)

In [11]:
item_dict =item_info.set_index('pum_id').to_dict()['pum_name']

In [13]:
item_info = pd.read_csv('../rawdata/item_info.csv')
all_df['pum_name'] =all_df['pum_id'].map(item_dict)

In [16]:
# 총 73종류
all_df.pum_name.value_counts()

pum_name
유아복      110676
회화용구      94600
아동화       87748
필기구       75004
스웨터       67605
          ...  
탄산음료       3681
이유식        3619
김          3332
북어채        3306
기초화장품       159
Name: count, Length: 73, dtype: int64

In [18]:
# 구조 : good_name 을 활용해 pum_name 을 분류

all_df.head()

,collect_day,good_id,pum_id,pum_name,good_name,sales_price,discount_price,benifit
0,2014-01-01,3740.0,E061100,습기제거제,알뜰상품_제습제_4입,4650.0,NaN,NaN
1,2014-01-01,6847.0,L012040,치약,비손_예스미백치약_120G,770.0,NaN,NaN
2,2014-01-01,156664.0,L012080,화장지,해피트리_3겹천연펄프화장지_35M*12R(S),5800.0,NaN,NaN
3,2014-01-01,269074.0,L012180,NaN,리스테린 쿨 민트/후레쉬버스트/내추럴 시트러스/티쓰앤껌 750mlx2개 초특가,7800.0,NaN,NaN
4,2014-01-01,356548.0,C021010,아동화,Allie Ballet Flat,57140.0,NaN,NaN


In [22]:
idx2pum = {idx: pum_id for idx, pum_id in enumerate(all_df['pum_id'].unique())}
pum2idx = {v : k for k, v in idx2pum.items()}

In [25]:
all_df['pid'] = all_df['pum_id'].map(pum2idx)

In [50]:
rawdata = all_df[['good_name','pid']]

In [51]:
rawdata.head()

,good_name,pid
0,알뜰상품_제습제_4입,0
1,비손_예스미백치약_120G,1
2,해피트리_3겹천연펄프화장지_35M*12R(S),2
3,리스테린 쿨 민트/후레쉬버스트/내추럴 시트러스/티쓰앤껌 750mlx2개 초특가,3
4,Allie Ballet Flat,4


# 기본 한국어분류모델을 활용해 분류 baseline 을 생성하기

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim

In [103]:
!pip install transformers[pytorch]

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
from  transformers import AutoModelForSequenceClassification , AutoTokenizer
from transformers import pipeline

In [ ]:
model_name = 'bert-base-multilingual-cased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels = rawdata.pid.nunique())

In [98]:
class mydataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        token = [tokenizer(t, padding = 'max_length', max_length = 96,truncation = True, return_tensors = 'pt') for t in self.X[idx]]
        label = torch.tensor(self.y[idx])
        return {'feature': token, 'label': label}

In [99]:
from sklearn.model_selection import train_test_split

In [100]:
X_train, X_test, y_train, y_test =train_test_split(rawdata['good_name'].values, rawdata['pid'].values, test_size = 0.3, random_state = 1205)

In [101]:
X_train_ds = mydataset(X_train, y_train)
X_test_ds = mydataset(X_test, y_test)

In [96]:
X_train_ds[0]

TypeError: 'BertTokenizer' object is not callable

In [ ]:
DataLoader(X_train_ds)